In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor

In [ ]:
train=pd.read_csv('../input/covid19-global-forecasting-week-4/train.csv')
test=pd.read_csv('../input/covid19-global-forecasting-week-4/test.csv')
submission=pd.read_csv('../input/covid19-global-forecasting-week-4/submission.csv')

In [ ]:
train.head()

In [ ]:
len(train.Province_State.value_counts())

In [ ]:
test.head()

In [ ]:
submission.head()

We will use to Province_State,Country_Region,Date, and Fatalities predict ConfirmedCases
and will use Province_State,Country_Region,Date,and ConfirmedCases to predict  Fatalities

In [ ]:
all_data=pd.concat([train.drop(['ConfirmedCases','Fatalities'], axis =1), test])

In [ ]:
all_data.drop(['ForecastId'], axis=1, inplace=True)
all_data.head()


In [ ]:
print("there are ", str(len(train)),' for training, and total with test there are ', str(len(all_data))," test only are: ", str(len(test)))

Both

In [ ]:
all_data.isnull().sum()

Id wouldn't matter for our analysis, 

**Province_State** </br>
There are 133 different Province_State

In [ ]:
all_data['Province_State'].value_counts()

In [ ]:
fig=px.histogram(all_data,'Province_State')
fig.show()

The Province_State, they have equal count for each, making the mode not making sense.</br>
we can assume that for each country that provinence state is in capital

In [ ]:
fig=px.histogram(all_data,'Country_Region')
fig.show()

There is no obvious way to fill all missing values in Province_State

In [ ]:
all_data['Province_State'].fillna(" ", inplace=True)

In [ ]:
all_data['Country_prov']=all_data['Country_Region']+" "+all_data['Province_State']
del all_data['Country_Region']
del all_data['Province_State']

In [ ]:
all_data.head()

In [ ]:
all_data['Country_prov'].value_counts()

## Date </br>
 for the date, there are two options, one is to extract day, month, and year second is to get the unix value for that date.</br>
 i have no prefrenece i will try both and see which gives better performace             


In [ ]:
all_data_date=all_data.copy(deep=True)
all_data_unix=all_data.copy(deep=True)

**all_data date** </br>
sample 2020-04-02 --> Day :2 , month : 4 , year **2020**


In [ ]:
def split_date(date):
    date = date.split('-')
    date[0] = int(date[0])
    if(date[1][0] == '0'):
        date[1] = int(date[1][1])
    else:
        date[1] = int(date[1])
    if(date[2][0] == '0'):
        date[2] = int(date[2][1])
    else:
        date[2] = int(date[2])    
    return date
all_data_date.Date=all_data.Date.apply(split_date)

In [ ]:
all_data.head()

In [ ]:
all_data_date['year']=all_data_date['Date'].apply(lambda x: x[0])
all_data_date['month']=all_data_date['Date'].apply(lambda x: x[1])
all_data_date['day']=all_data_date['Date'].apply(lambda x: x[2])
all_data_date.drop(['Date'], axis=1, inplace=True)

In [ ]:
all_data_date.head()

**all_data unix** </br>

In [ ]:
import time
import datetime
def date_to_unix(time_given):
  return int(time.mktime(datetime.datetime.strptime(time_given, "%Y-%m-%d").timetuple()))


In [ ]:
all_data_unix['Date_unix']=all_data_unix['Date'].apply(lambda x: date_to_unix(x))
all_data_unix.drop(['Date'],axis=1, inplace=True)

In [ ]:
all_data_unix.drop(['Id'], axis=1, inplace=True)
all_data_date.drop(['Id'], axis=1, inplace=True)

In [ ]:
all_data_unix.head()

In [ ]:
all_data_date.head()

In [ ]:
all_data_date.isnull().sum()


In [ ]:
all_data_unix.isnull().sum()

## ConfirmedCases

In [ ]:
train_date=all_data_date.head(len(train))
train_unix=all_data_unix.head(len(train))
#train_date['Fatalities']=train['Fatalities']
#train_unix['Fatalities']=train['Fatalities']
train_date['ConfirmedCases']=train['ConfirmedCases']
train_unix['ConfirmedCases']=train['ConfirmedCases']

In [ ]:
train_date.head()

In [ ]:
train_unix.head()

In [ ]:
all_data['Country_prov'].value_counts()

In [ ]:
sns.heatmap( train_unix.corr(), annot=True, fmt='0.2f')

In [ ]:
sns.heatmap( train_date.corr(), annot=True, fmt='0.2f')

We see that there are big correlation nearly 1, with confirmedCases adn Fatalities

In [ ]:
train_date.info()

# Scalling


In [ ]:
train_unix.head()

In [ ]:
train_date_num=train_date[['year','month','day']]
train_unix_num=train_unix[['Date_unix']]

scaler_date=MinMaxScaler().fit(train_date[['year','month','day']])
scaler_unix=MinMaxScaler().fit(train_unix[['Date_unix']])

train_date_num=scaler_date.transform(train_date_num)
train_unix_num=scaler_unix.transform(train_unix_num)

In [ ]:
#train_date[['year','month','day','Fatalities']]=train_date_num
#train_unix[['Date_unix','Fatalities']]=train_unix_num
train_date[['year','month','day']]=train_date_num
train_unix[['Date_unix']]=train_unix_num

In [ ]:
train_date.head()

In [ ]:
train_unix.head()

# OneHotEncoder

In [ ]:
train_date['Country_prov'].value_counts()

In [ ]:
train_date_cat=train_date[['Country_prov']]
train_unix_cat=train_unix[['Country_prov']]
le_date = preprocessing.OneHotEncoder()
le_unix = preprocessing.OneHotEncoder()
train_date_hot=le_date.fit_transform(train_date_cat)
train_unix_hot=le_unix.fit_transform(train_unix_cat)
train_date_hot.toarray()



In [ ]:
train_date_hot

In [ ]:
type(le_date.categories_[0])

In [ ]:
train_date.shape

In [ ]:
train_date[le_date.categories_[0]]=train_date_hot.toarray()

In [ ]:
train_unix[le_unix.categories_[0]]=train_unix_hot.toarray()

In [ ]:
train_date.drop(['Country_prov'], axis=1, inplace=True)
train_unix.drop(['Country_prov'], axis=1, inplace=True)

In [ ]:
train_date.head()

# Splitting


In [ ]:
x_train_date, x_test_date, y_train_date,y_test_date=train_test_split(train_date.drop(['ConfirmedCases'], axis=1), train_date['ConfirmedCases'],test_size=0.4 )
x_val_date,x_test_date,y_val_date, y_test_date =train_test_split(x_test_date,y_test_date, test_size=0.5)

x_train_unix, x_test_unix, y_train_unix,y_test_unix=train_test_split(train_unix.drop(['ConfirmedCases'], axis=1), train_unix['ConfirmedCases'],test_size=0.4 )
x_val_unix,x_test_unix,y_val_unix, y_test_unix =train_test_split(x_test_unix,y_test_unix, test_size=0.5)



**ElasticNet Regression**

In [ ]:
ridge_date=ElasticNet(alpha=0.1,l1_ratio = 0.9)
ridge_unix=ElasticNet(alpha=0.1,l1_ratio = 0.9)

In [ ]:
ridge_date.fit(x_train_date,y_train_date)
ridge_unix.fit(x_train_unix, y_train_unix)

In [ ]:
ridge_date.score(x_val_date, y_val_date)


In [ ]:
ridge_unix.score(x_val_unix, y_val_unix)


## LAsso


Trying corss validation with LinearRegression

In [ ]:
lasso = Lasso()

print(cross_val_score(lasso,x_val_date, y_val_date,cv=10))


I find lasso gives better estimes so i will use lasso

In [ ]:
lasso_date=Lasso()
lasso_unix=Lasso()

In [ ]:
lasso_date.fit(x_train_date,y_train_date)
lasso_unix.fit(x_train_unix, y_train_unix)

In [ ]:
lasso_date.score(x_val_date, y_val_date)


In [ ]:
lasso_unix.score(x_val_unix, y_val_unix)


Seems that lasso performs better, and unix is also better

In [ ]:
lasso_date.score(x_test_date, y_test_date)


In [ ]:
lasso_unix.fit(train_unix.drop(['ConfirmedCases'], axis=1), train_unix['ConfirmedCases'])

it seems date is performing better, so i will go with it.

# predicting confirmed case 

In [ ]:
test.head()

**get the test look like train**

In [ ]:
test_date=test.copy(deep=True)
test_date.fillna(" ", inplace=True)
test_date['Country_prov']=test_date['Country_Region']+" "+test_date['Province_State']
del test_date['Country_Region']
del test_date['Province_State']

test_date.Date=test_date.Date.apply(split_date)
test_date['year']=test_date['Date'].apply(lambda x: x[0])
test_date['month']=test_date['Date'].apply(lambda x: x[1])
test_date['day']=test_date['Date'].apply(lambda x: x[2])
test_date.drop(['Date'], axis=1, inplace=True)

In [ ]:
test_date.head()

In [ ]:
test_date_num=test_date[['year','month','day']]

test_date_num=scaler_date.transform(test_date_num)
test_date[['year','month','day']]=test_date_num


In [ ]:
test_date.head()

In [ ]:

test_date.isnull().sum()

In [ ]:
test_date_cat=test_date[['Country_prov']]
test_date_hot=le_date.transform(test_date_cat)
test_date[le_date.categories_[0]]=test_date_hot.toarray()
test_date.drop(['Country_prov'], axis=1, inplace=True)
test_date.drop(['ForecastId'], axis=1, inplace=True)



In [ ]:
test_date.head()

In [ ]:
y_pred_confirmed=lasso_date.predict(test_date.values)

## predicting Fatalities

In [ ]:
all_data_date.head()

In [ ]:
train_date_fac=all_data_date.head(len(train))
train_date_fac['Fatalities']=train['Fatalities']


In [ ]:
train_date_fac.head()

In [ ]:
train_date_fac_num=train_date_fac[['year','month','day']]

scaler_date_fac=MinMaxScaler().fit(train_date_fac[['year','month','day']])

train_date_fac_num=scaler_date_fac.transform(train_date_fac_num)
train_date_fac[['year','month','day']]=train_date_fac_num


In [ ]:
train_date_fac.head()

In [ ]:
train_date_fac_cat=train_date_fac[['Country_prov']]

le_date_cat = preprocessing.OneHotEncoder()

train_date_hot_cat=le_date_cat.fit_transform(train_date_fac_cat)


In [ ]:
train_date_fac[le_date_cat.categories_[0]]=train_date_hot_cat.toarray()

In [ ]:
train_date_fac.drop(['Country_prov'], axis=1, inplace=True)
train_date_fac.head()


In [ ]:
x_train_date, x_test_date, y_train_date,y_test_date=train_test_split(train_date_fac.drop(['Fatalities'], axis=1), train_date_fac['Fatalities'],test_size=0.4 )
x_val_date,x_test_date,y_val_date, y_test_date =train_test_split(x_test_date,y_test_date, test_size=0.5)

In [ ]:
lasso_date_fac=Lasso()
lasso_date_fac.fit(x_train_date,y_train_date)


In [ ]:
lasso_date_fac.score(x_val_date, y_val_date)


In [ ]:
y_pred_fac=lasso_date_fac.predict(test_date.values)

In [ ]:
submission['ConfirmedCases'] = y_pred_confirmed
submission['Fatalities'] = y_pred_fac

In [ ]:
submission.to_csv("submission.csv" , index = False)

In [ ]:
submission.head()